In [ ]:
import glob
import os
import shutil

from astropy import stats
import numpy as np
import pandas as pd

In [ ]:
postflash_data = pd.read_pickle('/grp/hst/wfc3u/postflash_2021/all_raw_postflash_data.pkl')

In [ ]:
postflash_data[0:10]

In [ ]:
low_sub_pf = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True)] 
subarray_pf = postflash_data.loc[(postflash_data['subarray'] == True)] 
fullframe_pf = postflash_data.loc[(postflash_data['subarray'] == False)] 
low_sub_pf_A = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A')] 
low_sub_pf_B = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B')] 
paths = subarray_pf.path.tolist()
print (paths)

In [ ]:
means = []
sig_means = []
medians = []
sig_medians = []
stds = []
sig_stds = []

for f in paths:
    path = f
    print(path)
    data = fits.getdata(path)
    data_clip = stats.sigma_clip(data, 3)
    
    mean = np.mean(data)
    sig_mean = np.mean(data_clip)
    median = np.median(data)
    sig_median = np.median(data_clip)
    std = np.std(data)
    sig_std = np.std(data_clip)
    
    
    means.append(mean)
    sig_means.append(sig_mean)
    medians.append(median)
    sig_medians.append(sig_median)
    stds.append(std)
    sig_stds.append(sig_std)

dict = {'path': paths, 'mean': means, 'sig_mean': sig_means,'median': medians, 'sig_median': sig_medians, 'std': stds, 'sig_std': sig_stds}
path_and_stats_sub = pd.DataFrame(dict)

In [ ]:
subarray_mean = pd.merge(left=subarray_pf, right=path_and_mean_sub, left_on='path', right_on='path').set_index(subarray_pf.index)

In [ ]:
subarray_mean.to_pickle('/grp/hst/wfc3u/postflash_2021/all_sub_pf_and_mean_data.pkl')
subarray_mean = pd.read_pickle('/grp/hst/wfc3u/postflash_2021/all_sub_pf_and_mean_data.pkl')
print (subarray_mean)